In [ ]:
!pip3 install pprint
from pprint import pprint

In [3]:
!pip3 install beautifulsoup4

     |████████████████████████████████| 112kB 2.6MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
!pip3 install --upgrade pip

     |████████████████████████████████| 1.5MB 1.7MB/s eta 0:00:01
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip3 install requests

  Using cached requests-2.23.0-py2.py3-none-any.whl (58 kB)
  Using cached idna-2.9-py2.py3-none-any.whl (58 kB)
  Using cached certifi-2020.4.5.1-py2.py3-none-any.whl (157 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached urllib3-1.25.9-py2.py3-none-any.whl (126 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from six.moves.urllib.parse import quote as _quote
from bs4 import BeautifulSoup as _BeautifulSoup
import requests as _requests

__BASE_URL__ = 'https://lyrics.wikia.com'

In [2]:
class LyricsNotFound(Exception):
    __module__ = Exception.__module__

    def __init__(self, message=None):
        super(LyricsNotFound, self).__init__(message)

In [3]:
def urlize(string):
    """Convert string to LyricWikia format"""
    return _quote('_'.join(string.title().split()))

In [4]:
def create_url(artist, song):
    """Create the URL in the LyricWikia format"""
    return (__BASE_URL__ +
            '/wiki/{artist}:{song}'.format(artist=urlize(artist),
                                           song=urlize(song)))

In [5]:
def get_lyrics(artist, song, linesep='\n', timeout=None):
    """Retrieve the lyrics of the song and return the first one in case
    multiple versions are available."""
    return get_all_lyrics(artist, song, linesep, timeout)[0]

In [27]:

def get_all_lyrics(artist, song, linesep=' \n ', timeout=None):
    """Retrieve a list of all the lyrics versions of a song."""
    url = create_url(artist, song)
    response = _requests.get(url, timeout=timeout)
    soup = _BeautifulSoup(response.content, "html.parser")
    lyricboxes = soup.findAll('div', {'class': 'lyricbox'})

    if not lyricboxes:
        raise LyricsNotFound('Cannot download lyrics')

    for lyricbox in lyricboxes:
        for br in lyricbox.findAll('br'):
            br.replace_with(linesep)

    return [lyricbox.text.strip() for lyricbox in lyricboxes]

In [7]:
class Song(object):
    """A Song backed by the LyricWikia API"""

    def __init__(self, artist, title):
        self.artist = artist
        self.title = title

    @property
    def lyrics(self):
        """Song lyrics obtained by parsing the LyricWikia page"""
        return get_lyrics(self.artist, self.title)

    def __str__(self):
        return "Song(artist='%s', title='%s')" % (self.artist, self.title)

    def __repr__(self):
        return str(self)

In [8]:
class Album(object):
    """An Album backed by the LyricWikia API"""

    def __init__(self, artist, album_data):
        self.artist = artist
        self.title = album_data['album']
        self.year = album_data['year']
        self.songs = [Song(artist, song) for song in album_data['songs']]

    def __str__(self):
        return "Album(artist='%s', title='%s')" % (self.artist, self.title)

    def __repr__(self):
        return str(self)

In [9]:

class Artist(object):
    """An Artist backed by the LyricWikia API"""

    __API__ = __BASE_URL__ + '/api.php?fmt=json&func=getArtist&artist={artist}'

    def __init__(self, name):
        url = self.__API__.format(artist=urlize(name))
        data = _requests.get(url).json()
        self.name = data['artist']
        self.albums = [Album(self.name, album) for album in data['albums']]

    def __str__(self):
        return "Artist(name='%s')" % (self.name)

    def __repr__(self):
        return str(self)

In [25]:
lyrics = get_lyrics('Avicii','Waiting For Love')

In [26]:
pprint (lyrics)

("Where there's a will, there's a way, kinda beautifulAnd every night has its "
 "day, so magicalAnd if there's love in this life, there's no obstacleThat "
 "can't be defeatedFor every tyrant a tear for the vulnerableIn every lost "
 "soul the bones of a miracleFor every dreamer a dream we're unstoppableWith "
 'something to believe inMonday left me brokenTuesday I was through with '
 'hopingWednesday my empty arms were openThursday waiting for love, waiting '
 "for loveThank the stars it's FridayI'm burning like a fire gone wild on "
 "SaturdayGuess I won't be coming to church on SundayI'll be waiting for love, "
 'waiting for loveTo come aroundWe are one of a kind irreplaceableHow did I '
 "get so blind and so cynicalIf there's love in this life we're unstoppableNo "
 "we can't be defeatedMonday left me brokenTuesday I was through with "
 'hopingWednesday my empty arms were openThursday waiting for love, waiting '
 "for loveThank the stars it's FridayI'm burning like a fire gone wil